In [4]:
from aocd.models import Puzzle

def parses(text):
    def parse_arr(s):
        arr = []
        for row in s.split('/'):
            arr.append([v == '#' for v in row])
        return np.array(arr, dtype=np.uint8)
    lines = [line.split(' => ') for line in text.strip().split('\n')]
    return [(parse_arr(i), parse_arr(j)) for i, j in lines]

puzzle = Puzzle(year=2017, day=21)
data = parses(puzzle.input_data)

In [5]:
    sample = parses("""../.# => ##./#../...
    .#./..#/### => #..#/..../..../#..#""")

In [6]:
def transformations(arr):
    orientations = []
    for x in (arr, np.fliplr(arr)):
        for k in range(4):
            orientations.append(np.rot90(x, k))
    return orientations

def rothash(arr):
    return sum([hash(t.tobytes()) for t in transformations(arr)])

In [12]:
from einops import rearrange

In [13]:
def solve_a(data, steps):
    mapping = {}
    tile = np.array([[0,1,0],[0,0,1],[1,1,1]], dtype=np.uint8)
    
    for a, b in data:
        mapping[rothash(a)] = b
    
    for i in range(steps):
        N = len(tile)
        k = 2 if N % 2 == 0 else 3
        tiles = rearrange(tile, '(h h1) (w w1) -> h w h1 w1', h1=k, w1=k)
        transformed_tiles = np.array([[mapping[rothash(tile)] for tile in row] for row in tiles])
        tile = rearrange(transformed_tiles, 'h w h1 w1 -> (h h1) (w w1)')
#         print(tile.sum())
        print(tile.shape)
#         render(tile)
    
    return tile.sum()

In [14]:
def render(arr):
    s = ''
    for row in arr:
        for x in row:
            s += '#' if x else '.'
        s += '\n'
    print(s)

In [15]:
solve_a(sample, 2)

(4, 4)
(6, 6)


12

In [17]:
solve_a(data, 6)

(4, 4)
(6, 6)
(9, 9)
(12, 12)
(18, 18)
(27, 27)


299

In [71]:
def solve_b(data, steps):
    assert steps % 3 == 0
    mapping = {}
    
    for a, b in data:
        mapping[rothash(a)] = b
        
    counts = {}
    sums = {}
    for tile, _ in data:
        if tile.shape == (3,3):
            sums[rothash(tile)] = int(tile.sum())
            tile_4x4 = mapping[rothash(tile)]
            tiles_4_2x2 = rearrange(tile_4x4, '(h h1) (w w1) -> h w h1 w1', h1=2, w1=2)
            tiles_4_3x3 = np.array([[mapping[rothash(tile)] for tile in row] for row in tiles_4_2x2])
            tile_6x6 = rearrange(tiles_4_3x3, 'h w h1 w1 -> (h h1) (w w1)')
            tiles_9_2x2 = rearrange(tile_6x6, '(h h1) (w w1) -> h w h1 w1', h1=2, w1=2)
            tiles_9_3x3 = np.array([[mapping[rothash(tile)] for tile in row] for row in tiles_9_2x2])
            counts[rothash(tile)] = Counter([rothash(t) for t in tiles_9_3x3.reshape(-1, 3, 3)])
    
    tile = np.array([[0,1,0],[0,0,1],[1,1,1]], dtype=np.uint8)
    state = {rothash(tile): 1}
    for i in range(steps//3):
        new_state = defaultdict(int)
        for tile, n in state.items():
            for new_tile, m in counts[tile].items():
                new_state[new_tile] += n * m
        state = new_state
    return sum([sums[t] * n for t, n in state.items()])

In [73]:
solve_b(data, 18)

1984683

In [48]:
sum([1,2,3.])

6.0

In [101]:
# for i in range(4):
#     print(sum(sorted([hash(r.tobytes()) for r in transformations(np.rot90(x,i))])))

In [78]:
x = sample[0][1]

In [80]:
# transformations(x)

In [61]:
sample[0][0]

array([[False, False],
       [False,  True]])

In [62]:
np.rot90(sample[0][0], 0)

array([[False, False],
       [False,  True]])

In [93]:
rothash(np.rot90(x, 0))

-4528512049455049600

In [94]:
rothash(np.rot90(x, 1))

-29083207993740643328

In [49]:
orientations(sample[0][0])

[array([[False, False],
        [False,  True]]),
 array([[False,  True],
        [False, False]]),
 array([[ True, False],
        [False, False]]),
 array([[False, False],
        [ True, False]]),
 array([[False, False],
        [ True, False]]),
 array([[False, False],
        [False,  True]]),
 array([[False,  True],
        [False, False]]),
 array([[ True, False],
        [False, False]])]

In [34]:
# class Tile:
    
#     def __init__(self, arr):
#         self.arr = arr
#         self.hash = self.compute_hash(self)
        
#     @property
#     def orientations(self):
#         orientations = []
#         for x in (self.arr, np.fliplr(self.arr)):
#             for k in range(3):
#                 orientations.append(np.rot90(x, k))
#         return orientations
    
#     def compute_hash(self):
#         return hash((Tile._render(o) for o in self.orientations))

#     @staticmethod
#     def fromstr(s):
#         arr = [[i]]
#         return Tile(np.array(arr, dtype=bool))
    
#     def render(self):
#         return Tile._render(self.arr)
    
#     @staticmethod
#     def _render(arr):
#         s = ""
#         for row in arr:
#             for val in row:
#                 s += '.#'[val]
#             s += '\n'
#         return s
        
#     def __hash__(self):
#         return self.compute_hash()        